### libraries import

In [1]:
import os

#gradio interface
import gradio as gr

from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

#STT (speech to text)
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

#TTS (text to speech)
import torch
from TTS.api import TTS
from IPython.display import Audio

#json request for APIs
import requests
import json

#regular expressions
import re

#langchain and function calling
from typing import List, Literal, Union
import requests
from functools import partial
from geopy.geocoders import Nominatim
import math


#langchain, not used anymore since I had to find another way fast to stop using the endpoint, but could be interesting to reuse 
from langchain.tools.base import StructuredTool
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.prompts import StringPromptTemplate
from langchain.llms import HuggingFaceTextGenInference
from langchain.chains import LLMChain



from datetime import datetime, timedelta, timezone
from transformers import pipeline
import inspect

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
os.environ["COQUI_TOS_AGREED"] = "1"

### Models loads

In [3]:
# load model and processor for speech-to-text
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
modelw = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
modelw.config.forced_decoder_ids = None

#load model for text to speech
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1.1").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 > tts_models/multilingual/multi-dataset/xtts_v1.1 is already downloaded.


/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


 > Using model: xtts


In [4]:
#load model language recognition
model_ckpt = "papluca/xlm-roberta-base-language-detection"
pipe_language = pipeline("text-classification", model=model_ckpt)


In [5]:
#load model llama2
mn = 'stabilityai/StableBeluga-7B' #mn = "TheBloke/Llama-2-7b-Chat-GPTQ" --> other possibility 
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_4bit=True) #torch_dtype=torch.float16
tokr = AutoTokenizer.from_pretrained(mn, load_in_4bit=True) #tokenizer

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


In [6]:
#NexusRaven for function calling
model_id = "Nexusflow/NexusRaven-13B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
modelNexus = AutoModelForCausalLM.from_pretrained(model_id, device_map=0, load_in_4bit=True)
pipe = pipeline("text-generation", model=modelNexus, tokenizer = tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.36s/it]


## Function calling with NexusRaven 

In [ ]:
# load api key from .env file
# weather api and tomtom api key
from dotenv import load_dotenv
load_dotenv()
weather_api_key = os.getenv("WEATHER_API_KEY")
tomtom_api_key = os.getenv("TOMTOM_API_KEY")
print(weather_api_key, tomtom_api_key)

In [81]:
#FUNCTION CALLING 

#API keys
TOMTOM_KEY= "your_key" 
WHEATHER_API_KEY = "your_key" 

##########################################################
# Step 1: Define the functions you want to articulate. ###
##########################################################

########################################################################################
# Functions called  in the articulated functions (not directly called by the model): ###
########################################################################################

geolocator = Nominatim(user_agent="MyApp")

def find_precise_place(lat, lon):
    location = geolocator.reverse(str(lat) +", " + str(lon))
    return location.raw.get('display_name', {})

def find_coordinates(address):
    coord = geolocator.geocode(address)
    lat = coord.latitude
    lon = coord.longitude
    return(lat,lon)


def check_city_coordinates(lat = "", lon = "", city = "", **kwargs):
    """
    :param lat: latitude
    :param lon: longitude
    :param city: name of the city

    Checks if the coordinates correspond to the city, if not update the coordinate to correspond to the city
    """
    if lat != "0" and lon != "0":
        reverse = partial(geolocator.reverse, language="en")
        location = reverse(f"{lat}, {lon}")
        address = location.raw.get('address', {})
        city = address.get('city') or address.get('town') or address.get('village') or address.get('county')
    else : 
        reverse = partial(geolocator.reverse, language="en")
        location = reverse(f"{lat}, {lon}")
        address = location.raw.get('address', {})
        city_name = address.get('city') or address.get('town') or address.get('village') or address.get('county')
        if city_name is None :
            city_name = 'not_found'
        print(city_name)
        if city_name.lower() != city.lower():
            coord = geolocator.geocode(city )
            if coord is None:
                coord = geolocator.geocode(city)
            lat = coord.latitude
            lon = coord.longitude
    return lat, lon, city

# Select coordinates at equal distance, including the last one
def select_equally_spaced_coordinates(coords, number_of_points=10):
    n = len(coords)
    selected_coords = []
    interval = max((n - 1) / (number_of_points - 1), 1)
    for i in range(number_of_points):
        # Calculate the index, ensuring it doesn't exceed the bounds of the list
        index = int(round(i * interval))
        if index < n:
            selected_coords.append(coords[index])
    return selected_coords

###################################################
# Functions we want to articulate (APIs calls): ###
###################################################

def search_along_route(latitude_depart, longitude_depart, city_destination, type_of_poi):
    """
    Return some of the closest points of interest along the route from the depart point, specified by its coordinates and a city destination.
    :param latitude_depart (string):  Required. Latitude of depart location
    :param longitude_depart (string):  Required. Longitude of depart location
    :param city_destination (string): Required. City destination
    :param type_of_poi (string): Required. type of point of interest depending on what the user wants to do.
    """
    
    lat_dest, lon_dest = find_coordinates(city_destination)
    print(lat_dest)
    
    r = requests.get('https://api.tomtom.com/routing/1/calculateRoute/{0},{1}:{2},{3}/json?key={4}'.format(
                        latitude_depart,
                        longitude_depart,
                        lat_dest,
                        lon_dest,
                        TOMTOM_KEY
    ))
    
    coord_route = select_equally_spaced_coordinates(r.json()['routes'][0]['legs'][0]['points'])

    # The API endpoint for searching along a route
    url = f'https://api.tomtom.com/search/2/searchAlongRoute/{type_of_poi}.json?key={TOMTOM_KEY}&maxDetourTime=700&limit=20&sortBy=detourTime'

    # The data payload
    payload = {
      "route": {
        "points": [
          {"lat": float(latitude_depart), "lon": float(longitude_depart)},
          {"lat": float(coord_route[1]['latitude']), "lon": float(coord_route[1]['longitude'])},
          {"lat": float(coord_route[2]['latitude']), "lon": float(coord_route[2]['longitude'])},
          {"lat": float(coord_route[3]['latitude']), "lon": float(coord_route[3]['longitude'])},
          {"lat": float(coord_route[4]['latitude']), "lon": float(coord_route[4]['longitude'])},
          {"lat": float(coord_route[5]['latitude']), "lon": float(coord_route[5]['longitude'])},
          {"lat": float(coord_route[6]['latitude']), "lon": float(coord_route[6]['longitude'])},
          {"lat": float(coord_route[7]['latitude']), "lon": float(coord_route[7]['longitude'])},
          {"lat": float(coord_route[8]['latitude']), "lon": float(coord_route[8]['longitude'])},
          {"lat": float(lat_dest), "lon": float(lon_dest)},
        ]
      }
    }

    # Make the POST request
    response = requests.post(url, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print(json.dumps(data, indent=4))
    else:
        print('Failed to retrieve data:', response.status_code)
    answer = ""
    for result in data['results']:
        name = result['poi']['name']
        address = result['address']['freeformAddress']
        detour_time = result['detourTime']
        answer = answer + f" \nAlong the route to {city_destination}, there is the {name} at {address} that would represent a detour of {int(detour_time/60)} minutes."
        
    return answer


def find_points_of_interest(lat="0", lon="0", city="", type_of_poi="restaurant", **kwargs):
    """
    Return some of the closest points of interest for a specific location and type of point of interest. The more parameters there are, the more precise.
    :param lat (string):  latitude
    :param lon (string):  longitude
    :param city (string): Required. city
    :param type_of_poi (string): Required. type of point of interest depending on what the user wants to do.
    """
    lat, lon, city = check_city_coordinates(lat,lon,city)

    r = requests.get(f'https://api.tomtom.com/search/2/search/{type_of_poi}'
                     '.json?key={0}&lat={1}&lon={2}&radius=10000&idxSet=POI&limit=100'.format(
                        TOMTOM_KEY,
                        lat,
                        lon
    ))

    # Parse JSON from the response
    data = r.json()
    #print(data)
    # Extract results
    results = data['results']

    # Sort the results based on distance
    sorted_results = sorted(results, key=lambda x: x['dist'])
    #print(sorted_results)

    # Format and limit to top 5 results
    formatted_results = [
        f"The {type_of_poi} {result['poi']['name']} is {int(result['dist'])} meters away"
        for result in sorted_results[:5]
    ]


    return ". ".join(formatted_results)

def find_route(lat_depart="0", lon_depart="0", city_depart="", address_destination="", depart_time ="", **kwargs):
    """
    Return the distance and the estimated time to go to a specific destination from the current place, at a specified depart time.
    :param lat_depart (string):  latitude of depart
    :param lon_depart (string):  longitude of depart
    :param city_depart (string): Required. city of depart
    :param address_destination (string): Required. The destination
    :param depart_time (string):  departure hour, in the format '08:00:20'.
    """
    print(address_destination)
    date = "2025-03-29T"
    departure_time = '2024-02-01T' + depart_time
    lat, lon, city = check_city_coordinates(lat_depart,lon_depart,city_depart)
    lat_dest, lon_dest = find_coordinates(address_destination)
    #print(lat_dest, lon_dest)
    
    #print(departure_time)

    r = requests.get('https://api.tomtom.com/routing/1/calculateRoute/{0},{1}:{2},{3}/json?key={4}&departAt={5}'.format(
                        lat_depart,
                        lon_depart,
                        lat_dest,
                        lon_dest,
                        TOMTOM_KEY,
                        departure_time
    ))

    # Parse JSON from the response
    data = r.json()
    #print(data)
    
    #print(data)
    
    result = data['routes'][0]['summary']

    # Calculate distance in kilometers (1 meter = 0.001 kilometers)
    distance_km = result['lengthInMeters'] * 0.001

    # Calculate travel time in minutes (1 second = 1/60 minutes)
    time_minutes = result['travelTimeInSeconds'] / 60
    if time_minutes < 60:
        time_display = f"{time_minutes:.0f} minutes"
    else:
        hours = int(time_minutes / 60)
        minutes = int(time_minutes % 60)
        time_display = f"{hours} hours" + (f" and {minutes} minutes" if minutes > 0 else "")
        
    # Extract arrival time from the JSON structure
    arrival_time_str = result['arrivalTime']

    # Convert string to datetime object
    arrival_time = datetime.fromisoformat(arrival_time_str)

    # Extract and display the arrival hour in HH:MM format
    arrival_hour_display = arrival_time.strftime("%H:%M")


    # return the distance and time
    return(f"The route to go to {address_destination} is {distance_km:.2f} km and {time_display}. Leaving now, the arrival time is estimated at {arrival_hour_display} " )

    
    # Sort the results based on distance
    #sorted_results = sorted(results, key=lambda x: x['dist'])

    #return ". ".join(formatted_results)

#current weather API
def get_weather(city_name:str= "", **kwargs):
    """
    Returns the CURRENT weather in a specified city.
    Args:
    city_name (string) : Required. The name of the city.
    """
    # The endpoint URL provided by WeatherAPI
    url = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={city_name}&aqi=no"

    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        weather_data = response.json()

        # Extracting the necessary pieces of data
        location = weather_data['location']['name']
        region = weather_data['location']['region']
        country = weather_data['location']['country']
        time = weather_data['location']['localtime']
        temperature_c = weather_data['current']['temp_c']
        condition_text = weather_data['current']['condition']['text']
        wind_mph = weather_data['current']['wind_mph']
        humidity = weather_data['current']['humidity']
        feelslike_c = weather_data['current']['feelslike_c']

        # Formulate the sentences
        weather_sentences = (
            f"The current weather in {location}, {region}, {country} is {condition_text} "
            f"with a temperature of {temperature_c}°C that feels like {feelslike_c}°C. "
            f"Humidity is at {humidity}%. "
            f"Wind speed is {wind_mph} mph."
        )
        return weather_sentences
    else:
        # Handle errors
        return f"Failed to get weather data: {response.status_code}, {response.text}"
    
#forecast API
def get_forecast(city_name:str= "", when = 0, **kwargs):
    """
    Returns the weather forecast in a specified number of days for a specified city .
    Args:
    city_name (string) : Required. The name of the city.
    when (int) : Required. in number of days (until the day for which we want to know the forecast) (example: tomorrow is 1, in two days is 2, etc.)
    """
    #print(when)
    when +=1
    # The endpoint URL provided by WeatherAPI
    url = f"http://api.weatherapi.com/v1/forecast.json?key={WEATHER_API_KEY}&q={city_name}&days={str(when)}&aqi=no"


    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Initialize an empty string to hold our result
        forecast_sentences = ""

        # Extract city information
        location = data.get('location', {})
        city_name = location.get('name', 'the specified location')
        
        #print(data)
    

        # Extract the forecast days
        forecast_days = data.get('forecast', {}).get('forecastday', [])[when-1:]
        #number = 0
        
        #print (forecast_days)

        for day in forecast_days:
            date = day.get('date', 'a specific day')
            conditions = day.get('day', {}).get('condition', {}).get('text', 'weather conditions')
            max_temp_c = day.get('day', {}).get('maxtemp_c', 'N/A')
            min_temp_c = day.get('day', {}).get('mintemp_c', 'N/A')
            chance_of_rain = day.get('day', {}).get('daily_chance_of_rain', 'N/A')
            
            if when == 1:
                number_str = 'today'
            elif when == 2:
                number_str = 'tomorrow'
            else:
                number_str = f'in {when-1} days'

            # Generate a sentence for the day's forecast
            forecast_sentence = f"On {date} ({number_str}) in {city_name}, the weather will be {conditions} with a high of {max_temp_c}°C and a low of {min_temp_c}°C. There's a {chance_of_rain}% chance of rain. "
            
            #number = number + 1
            # Add the sentence to the result
            forecast_sentences += forecast_sentence
        return forecast_sentences
    else:
        # Handle errors
        print( f"Failed to get weather data: {response.status_code}, {response.text}")
        return f'error {response.status_code}'


#############################################################
# Step 2: Let's define some utils for building the prompt ###
#############################################################


def format_functions_for_prompt(*functions):
    formatted_functions = []
    for func in functions:
        source_code = inspect.getsource(func)
        docstring = inspect.getdoc(func)
        formatted_functions.append(
            f"OPTION:\n<func_start>{source_code}<func_end>\n<docstring_start>\n{docstring}\n<docstring_end>"
        )
    return "\n".join(formatted_functions)


##############################
# Step 3: Construct Prompt ###
##############################


def construct_prompt(user_query: str, context):
    formatted_prompt = format_functions_for_prompt(get_weather, find_points_of_interest, find_route, get_forecast, search_along_route)
    formatted_prompt += f'\n\nContext : {context}'
    formatted_prompt += f"\n\nUser Query: Question: {user_query}\n"

    prompt = (
        "<human>:\n"
        + formatted_prompt
        + "Please pick a function from the above options that best answers the user query and fill in the appropriate arguments.<human_end>"
    )
    return prompt

#######################################
# Step 4: Execute the function call ###
#######################################


def execute_function_call(model_output):
    # Ignore everything after "Reflection" since that is not essential.
    function_call = (
        model_output[0]["generated_text"]
        .strip()
        .split("\n")[1]
        .replace("Initial Answer:", "")
        .strip()
    )

    try:
        return eval(function_call)
    except Exception as e:
        return str(e)


In [73]:
# might be deleted
# Compute a Simple equation
print("before everything: ")
!nvidia-smi
prompt = construct_prompt("What restaurants are there on the road from Luxembourg Gare, which coordinates are lat 49.5999681, lon 6.1342493, to Thionville?", "")
print("after creating prompt: ")
!nvidia-smi
model_output = pipe(
    prompt, do_sample=False, max_new_tokens=300, return_full_text=False
    )
print(model_output[0]["generated_text"])

print("creating the pipe of model output: ")
!nvidia-smi
result = execute_function_call(model_output)
print("execute function call: ")
!nvidia-smi
del model_output
import gc         # garbage collect library
gc.collect()
torch.cuda.empty_cache() 

#print("Model Output:", model_output)
print("Execution Result:", result)


#execute_function_call(pipe(construct_prompt("Is it raining in Belval, ?"), do_sample=False, max_new_tokens=300, return_full_text=False))

before everything: 
Tue Feb 27 12:54:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.02              Driver Version: 545.29.02    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0 Off |                  Off |
|  0%   46C    P8              24W / 450W |  15915MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


after creating prompt: 
Tue Feb 27 12:54:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.02              Driver Version: 545.29.02    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0 Off |                  Off |
|  0%   46C    P8              24W / 450W |  15915MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 
 Thought: The purpose of the def search_along_route(latitude_depart, longitude_depart, city_destination, type_of_poi) is to return some of the closest points of interest along the route from the depart point, specified by its coordinates and a city destination.
Initial Answer: search_along_route(49.5999681, 6.1342493, 'Thionville','restaurant')
Reflection: The search_along_route function takes in four arguments: latitude_depart, longitude_depart, city_destination, and type_of_poi.

The user has asked what restaurants are there on the road from Luxembourg Gare, which coordinates are lat 49.5999681, lon 6.1342493, to Thionville.

The call provided is search_along_route(49.5999681, 6.1342493, 'Thionville','restaurant').

The call can be improved because the function requires the latitude and longitude of the depart point, as well as the city destination. The call provided only provides the latitude and longitude of the depart point.

The correct call would be search_along_
creating the 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


49.3579272
{
    "summary": {
        "query": "restaurant",
        "queryType": "NON_NEAR",
        "queryTime": 542,
        "numResults": 20,
        "offset": 0,
        "totalResults": 20,
        "fuzzyLevel": 1,
        "geobiasCountry": "LU"
    },
    "results": [
        {
            "type": "POI",
            "id": "6iLTyW2L8iNsWvfZn-G3RA",
            "score": 2.9680526257,
            "dist": 6633.029897,
            "query": "restaurant",
            "info": "search:ta:442007000007898-LU",
            "poi": {
                "name": "Eat 'N' Break",
                "categorySet": [
                    {
                        "id": 7315
                    }
                ],
                "categories": [
                    "restaurant"
                ],
                "classifications": [
                    {
                        "code": "RESTAURANT",
                        "names": [
                            {
                                "nameLocal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## functions to process the anwser and the question

In [62]:
#generation of text with Stable beluga 
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

#to have a prompt corresponding to the specific format required by the fine-tuned model Stable Beluga
def mk_prompt(user, syst="### System:\nYou are a useful AI assistant in a car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and do not mention what you base your reply on.\n\n"): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [63]:
def car_answer_only(complete_answer, general_context):
    """returns only the AI assistant answer, without all context, to reply to the user"""
    pattern = r"Assistant:\\n(.*)(</s>|[.!?](\s|$))" #pattern = r"Assistant:\\n(.*?)</s>"

    match = re.search(pattern, complete_answer, re.DOTALL)

    if match:
        # Extracting the text
        model_answer = match.group(1)
        #print(complete_answer)
    else:
        #print(complete_answer)
        model_answer = "There has been an error with the generated response." 

    general_context +=  model_answer
    return (model_answer, general_context)
#print(model_answer)

In [74]:
def FnAnswer(general_context, ques, place, time, delete_history, state):
    """function to manage the two different llms (function calling and basic answer) and call them one after the other"""
    # Initialize state if it is None
    if delete_history == "Yes":
        state = None
    if state is None:
        conv_context = []
        conv_context.append(general_context)
        state = {}
        state['context'] = conv_context
        state['number'] = 0
        state['last_question'] = ""
        
    if type(ques) != str: 
        ques = ques[0]
        
    place = definePlace(place) #which on the predefined places it is
    
    formatted_context = '\n'.join(state['context'])
        
    #updated at every question
    general_context = f"""
    Recent conversation history: '{formatted_context}' (If empty, this indicates the beginning of the conversation).

    Previous question from the user: '{state['last_question']}' (This may or may not be related to the current question).

    User information: The user is inside a car in {place[0]}, with latitude {place[1]} and longitude {place[2]}. The user is mobile and can drive to different destinations. It is currently {time}

    """
    #first llm call (function calling model, NexusRaven)
    model_output= pipe(construct_prompt(ques, general_context), do_sample=False, max_new_tokens=300, return_full_text=False)
    call = execute_function_call(model_output) #call variable is formatted to as a call to a specific function with the required parameters
    print(call)
    #this is what will erase the model_output from the GPU memory to free up space
    del model_output
    import gc         # garbage collect library
    gc.collect()
    torch.cuda.empty_cache() 
        
    #updated at every question
    general_context += f'This information might be of help, use if it seems relevant, and ignore if not relevant to reply to the user: "{call}". '
    
    #question formatted for the StableBeluga llm (second llm), using the output of the first llm as context in general_context
    question=f"""Reply to the user and answer any question with the help of the provided context.

    ## Context

    {general_context} .

    ## Question

    {ques}"""

    complete_answer = str(gen(mk_prompt(question), 100)) #answer generation with StableBeluga (2nd llm)

    model_answer, general_context= car_answer_only(complete_answer, general_context) #to retrieve only the car answer 
    
    language = pipe_language(model_answer, top_k=1, truncation=True)[0]['label'] #detect the language of the answer, to modify the text-to-speech consequently
    
    state['last_question'] = ques #add the current question as 'last question' for the next question's context
    
    state['number']= state['number'] + 1  #adds 1 to the number of interactions with the car

    state['context'].append(str(state['number']) + '. User question: '+ ques + ', Model answer: ' + model_answer) #modifies the context
    
    #print("contexte : " + '\n'.join(state['context']))
    
    if len(state['context'])>5: #6 questions maximum in the context to avoid having too many information
        state['context'] = state['context'][1:]

    return model_answer, state['context'], state, language

In [75]:
def transcript(general_context, link_to_audio, voice, place, time, delete_history, state):
    """this function manages speech-to-text to input Fnanswer function and text-to-speech with the Fnanswer output"""
    # load audio from a specific path
    audio_path = link_to_audio
    audio_array, sampling_rate = librosa.load(link_to_audio, sr=16000)  # "sr=16000" ensures that the sampling rate is as required


    # process the audio array
    input_features = processor(audio_array, sampling_rate, return_tensors="pt").input_features


    predicted_ids = modelw.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    quest_processing = FnAnswer(general_context, transcription, place, time, delete_history, state)
    
    state=quest_processing[2]
    
    print("langue " + quest_processing[3])

    tts.tts_to_file(text= str(quest_processing[0]),
                file_path="output.wav",
                speaker_wav=f'Audio_Files/{voice}.wav',
                language=quest_processing[3],
                emotion = "angry")

    audio_path = "output.wav"
    return audio_path, state['context'], state

In [76]:
def definePlace(place):
    if(place == 'Luxembourg Gare, Luxembourg'):
        return('Luxembourg Gare', '49.5999681', '6.1342493' )
    elif (place =='Kirchberg Campus, Kirchberg'):
        return('Kirchberg Campus, Luxembourg', '49.62571206478235', '6.160082636815114')
    elif (place =='Belval Campus, Belval'):
        return('Belval-Université, Esch-sur-Alzette', '49.499531', '5.9462903')
    elif (place =='Eiffel Tower, Paris'):
        return('Eiffel Tower, Paris', '48.8582599', '2.2945006')
    elif (place=='Thionville, France'):
        return('Thionville, France', '49.357927', '6.167587')

## Interfaces (text and audio)

In [77]:
#INTERFACE WITH ONLY TEXT

# Generate options for hours (00-23) 
hour_options = [f"{i:02d}:00:00" for i in range(24)]

model_answer= ''
general_context= ''
# Define the initial state with some initial context.
print(general_context)
initial_state = {'context': general_context}
initial_context= initial_state['context']
# Create the Gradio interface.
iface = gr.Interface(
    fn=FnAnswer,
    inputs=[
        gr.Textbox(value=initial_context, visible=False),
        gr.Textbox(lines=2, placeholder="Type your message here..."),
        gr.Radio(choices=['Luxembourg Gare, Luxembourg', 'Kirchberg Campus, Kirchberg', 'Belval Campus, Belval', 'Eiffel Tower, Paris', 'Thionville, France'], label='Choose a location for your car', value= 'Kirchberg Campus, Kirchberg', show_label=True),
        gr.Dropdown(choices=hour_options, label="What time is it?", value = "08:00:00"),
        gr.Radio(["Yes", "No"], label="Delete the conversation history?", value = 'No'),
        gr.State()  # This will keep track of the context state across interactions.
    ],
    outputs=[
        gr.Textbox(),
        gr.Textbox(visible=False),
        gr.State()
    ]
)
gr.close_all()
# Launch the interface.
iface.launch(debug=True, share=True, server_name="0.0.0.0", server_port=7860)
#contextual=gr.Textbox(value=general_context, visible=False)
#demo = gr.Interface(fn=FnAnswer, inputs=[contextual,"text"], outputs=["text", contextual])

#demo.launch()


Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://0.0.0.0:7860


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/02/27 12:55:51 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


49.3579272
{
    "summary": {
        "query": "fastfood",
        "queryType": "NON_NEAR",
        "queryTime": 310,
        "numResults": 20,
        "offset": 0,
        "totalResults": 20,
        "fuzzyLevel": 1,
        "geobiasCountry": "LU"
    },
    "results": [
        {
            "type": "POI",
            "id": "J5-wTaFqeNCotY27tp8L2Q",
            "score": 2.7918899059,
            "dist": 29973.83986,
            "query": "fastfood",
            "info": "search:ta:250009013479571-FR",
            "poi": {
                "name": "O'Tacos",
                "categorySet": [
                    {
                        "id": 7315015
                    }
                ],
                "url": "www.facebook.com/Otacos-Thionville-1898601700407644/",
                "categories": [
                    "fast food",
                    "restaurant"
                ],
                "classifications": [
                    {
                        "code": "RESTAURANT",
  

...

the first one
The route to go to the first one is 634.64 km and 6 hours and 53 minutes. Leaving now, the arrival time is estimated at 14:53 


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The restaurant Eischen Paul Restaurant-Traiteur is 137 meters away. The restaurant Piri Piri Portuguese Restaurant & Bar is 149 meters away. The restaurant Sushi Shop is 165 meters away. The restaurant Vida Sushi Lounge-Kirchberg is 294 meters away. The restaurant Restaurant Himalaya 2 is 297 meters away


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


get_weather() takes from 0 to 1 positional arguments but 2 were given
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> None


In [69]:
#INTERFACE WITH AUDIO TO AUDIO

#to be able to use the microphone on chrome, you will have to go to chrome://flags/#unsafely-treat-insecure-origin-as-secure and enter http://10.186.115.21:7860/ 
#in "Insecure origins treated as secure", enable it and relaunch chrome

#example question: 
# what's the weather like outside?
# What's the closest restaurant from here?



# Generate options for hours (00-23) 
hour_options = [f"{i:02d}:00:00" for i in range(24)]

model_answer= ''
general_context= ''
# Define the initial state with some initial context.
print(general_context)
initial_state = {'context': general_context}
initial_context= initial_state['context']
# Create the Gradio interface.
iface = gr.Interface(
    fn=transcript,
    inputs=[
        gr.Textbox(value=initial_context, visible=False),
        gr.Audio( type='filepath', label = 'input audio'),
        gr.Radio(choices=['Donald Trump', 'Eddie Murphy'], label='Choose a voice', value= 'Donald Trump', show_label=True),  # Radio button for voice selection
        gr.Radio(choices=['Luxembourg Gare, Luxembourg', 'Kirchberg Campus, Kirchberg', 'Belval Campus, Belval', 'Eiffel Tower, Paris', 'Thionville, France'], label='Choose a location for your car', value= 'Kirchberg Campus, Kirchberg', show_label=True),
        gr.Dropdown(choices=hour_options, label="What time is it?", value = "08:00:00"),
        gr.Radio(["Yes", "No"], label="Delete the conversation history?", value = 'No'),
        gr.State()  # This will keep track of the context state across interactions.
    ],
    outputs=[
        gr.Audio(label = 'output audio'),
        gr.Textbox(visible=False),
        gr.State()
    ]
)
#close all interfaces open to make the port available
gr.close_all()
# Launch the interface.
iface.launch(debug=True, share=True, server_name="0.0.0.0", server_port=7860, ssl_verify=False)


Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://0.0.0.0:7860


KeyboardInterrupt: 

## Other possible APIs to use

In [80]:

def search_nearby(lat, lon, city, key):
    """
    :param lat: latitude
    :param lon: longitude
    :param key: api key
    :param type: type of poi
    :return: [5] results ['poi']['name']/['freeformAddress'] || ['position']['lat']/['lon']
    """
    results = []

    r = requests.get('https://api.tomtom.com/search/2/nearbySearch/.json?key={0}&lat={1}&lon={2}&radius=10000&limit=50'.format(
                        key,
                        lat,
                        lon
    ))

    for result in r.json()['results']:
        results.append(f"The {' '.join(result['poi']['categories'])} {result['poi']['name']} is {int(result['dist'])} meters far from {city}")
        if len(results) == 7:
            break

    return ". ".join(results)


print(search_nearby('49.625892805337514', '6.160417066963513', 'your location', TOMTOM_KEY))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)